In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
# 입력
trX = mnist.train.images.reshape(-1,28,28,1)
teX = mnist.test.images.reshape(-1,28,28,1)
trY = mnist.train.labels
teY = mnist.test.labels

In [18]:
batch_size = 128
test_size = 256

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


# placeholder
x = tf.placeholder('float', [None, 28, 28, 1])
y = tf.placeholder('float', [None, 10])
p_keep_conv = tf.placeholder('float')
p_keep_hidden = tf.placeholder('float')

                               
# filter
w = init_weights([3,3,1,32])       # 3*3*1 conv, 32 outputs(filter)
w2 = init_weights([3,3,32,64])     # 3*3*32 conv, 64 outputs
w3 = init_weights([3,3,64,128])    # 3*3*64 conv, 128 outputs


# convolutional neural net 
l1a = tf.nn.relu(tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME'))        # (?,28,28,32)
l1 = tf.nn.max_pool(l1a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')   # (?,14,14,32)
l1 = tf.nn.dropout(l1, p_keep_conv)

l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides=[1,1,1,1], padding='SAME'))      # (?,14,14,64)
l2 = tf.nn.max_pool(l2a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')   # (?,7,7,64)
l2 = tf.nn.dropout(l2, p_keep_conv)

l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides=[1,1,1,1], padding='SAME'))      # (?,7,7,128)
l3 = tf.nn.max_pool(l3a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')   # (?,4,4,128)


# fully connected net
w4 = init_weights([128*4*4, 625])    # FC 128*4*4 inputs, 625 outputs
w_o = init_weights([625, 10])        # FC 625 inputs, 10 outputs(labels)

l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])   # fc의 input으로 쓰기 위해서 쭉 늘어뜨림 (?,2048)
l3 = tf.nn.dropout(l3, p_keep_hidden)

l4 = tf.nn.relu(tf.matmul(l3,w4))
l4 = tf.nn.dropout(l4, p_keep_hidden)

pyx = tf.matmul(l4,w_o)


# cost & optimization
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pyx, y))
train_op = tf.train.RMSPropOptimizer(0.001,0.9).minimize(cost)
predict_op = tf.arg_max(pyx,1)


# train and test
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(20):
        training_batch = zip(range(0,len(trX), batch_size), 
                             range(batch_size,len(trX)+1, batch_size))  # (0,128), (128,256), (256,384), ...
        
        for start, end in training_batch:  
            sess.run(train_op, feed_dict={x:trX[start:end], y:trY[start:end],
                                         p_keep_conv:0.8, p_keep_hidden:0.5})
        
        test_indices = np.arange(len(teX))  # get a test batche
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]
        
        print(i, np.mean(np.argmax(teY[test_indices], axis=1)
                        == sess.run(predict_op, feed_dict={x:teX[test_indices],
                                                          y:teY[test_indices],
                                                          p_keep_conv:1.0,
                                                          p_keep_hidden:1.0})))
            
        

0 0.9453125
1 0.984375
2 0.98046875
3 0.98046875
4 0.98046875
5 0.99609375
6 0.9921875


KeyboardInterrupt: 